In [1]:
# Imports
import simpy
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
random.seed(76)

In [2]:
rand = random.randint(90,95)/100
PARROQUIA = 'BELLAVISTA'
TIPO_EDUCACION = 'REGULAR'
SOSTENIMIENTO = 'Particular'
MODALIDAD = 'Prescencial'
TOTAL_DOCENTES = round(11*0.9)
TOTAL_ESTUDIANTES = round(178*rand)
AULAS = 13
HORAS_DIA = 6
HORA_RECESO = 0.5
TOTAL_DIAS = HORAS_DIA * 60
AULAS_CERRADAS = 0

In [3]:
class Jornada(object):
    def __init__(self, environment, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad):
        self.env = environment
        self.aulas = aulas
        self.docentes = simpy.Resource(environment, docentes)
        self.estudiantes = estudiantes
        self.parroquia = parroquia
        self.tipo_educacion = tipo_educacion
        self.sostenimiento = sostenimiento
        self.modalidad = modalidad
    def curso(self):
        try:
            if(env.now < 180):
                print('Nuevo día iniciado a las: ', env.now)
                yield self.env.timeout(HORAS_DIA)
                print('Termina el día a las: ', env.now)
            else:
                print('Nuevo día iniciado a las: ', env.now)
                yield self.env.timeout(HORAS_DIA - HORA_RECESO)
                print('Empieza recreo a las: ', env.now)
                yield self.env.timeout(HORA_RECESO)
                print('Termina el día a las: ', env.now)
                self.prueba()
        except simpy.Interrupt:
            print("Interrupción")
            
    def prueba(self):
        global AULAS_CERRADAS
        estudiantes_prueba = round(TOTAL_ESTUDIANTES * 0.1)
        print('Se hace la prueba a ', estudiantes_prueba)
        foco_contagio = random.randint(1,100) + 2
        if(foco_contagio < 4):
            print('######## ESTUDIANTE CONTAGIADO, se cierra un curso ###########')
            self.aulas -= 1
            AULAS_CERRADAS += 1

In [4]:
def simulacion(env, jornada):
    with jornada.docentes.request() as profesor:
        yield profesor
        yield env.process(jornada.curso())

def ejecutar_simulacion(env, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad):
    while True:
        yield env.process(simulacion(env, Jornada(env, aulas, docentes, estudiantes, parroquia, tipo_educacion, sostenimiento, modalidad)))

In [10]:
print("Iniciando retorno a clases")
env = simpy.Environment()
env.process(ejecutar_simulacion(env, AULAS, TOTAL_DOCENTES, TOTAL_ESTUDIANTES, PARROQUIA, TIPO_EDUCACION, SOSTENIMIENTO, MODALIDAD))
env.run(until=TOTAL_DIAS)
print("")
print("")
print("--- RESULTADOS ---")
print("Total de cursos: ", AULAS)
print("Cursos Cerrados: ", AULAS_CERRADAS)
print("Docentes Disponibles: ", TOTAL_DOCENTES - AULAS_CERRADAS)

Iniciando retorno a clases
Nuevo día iniciado a las:  0
Termina el día a las:  6
Nuevo día iniciado a las:  6
Termina el día a las:  12
Nuevo día iniciado a las:  12
Termina el día a las:  18
Nuevo día iniciado a las:  18
Termina el día a las:  24
Nuevo día iniciado a las:  24
Termina el día a las:  30
Nuevo día iniciado a las:  30
Termina el día a las:  36
Nuevo día iniciado a las:  36
Termina el día a las:  42
Nuevo día iniciado a las:  42
Termina el día a las:  48
Nuevo día iniciado a las:  48
Termina el día a las:  54
Nuevo día iniciado a las:  54
Termina el día a las:  60
Nuevo día iniciado a las:  60
Termina el día a las:  66
Nuevo día iniciado a las:  66
Termina el día a las:  72
Nuevo día iniciado a las:  72
Termina el día a las:  78
Nuevo día iniciado a las:  78
Termina el día a las:  84
Nuevo día iniciado a las:  84
Termina el día a las:  90
Nuevo día iniciado a las:  90
Termina el día a las:  96
Nuevo día iniciado a las:  96
Termina el día a las:  102
Nuevo día iniciado a la